In [ ]:
pip install selenium

In [ ]:
%%javascript
function preventTimeout() {
    console.log("Preventing Colab from timing out...");
    document.querySelector("colab-connect-button#connect").click();
}
setInterval(preventTimeout, 60000);  // Clicks the "Connect" button every 60 seconds

<IPython.core.display.Javascript object>

In [ ]:
import time
import pandas as pd
import requests
import requests

from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RequestException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from datetime import datetime
from bs4 import BeautifulSoup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

def scrape_page(url, listing_type):
    try:
        url = 'https://www.dubizzle.com.lb/' + url
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-extensions')
        # options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

        browser = webdriver.Chrome(options=options)
        browser.get(url)

        wait = WebDriverWait(browser, 30)
        s_features = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Location']")))

        # Once fully loaded, retrieve the page source
        page_source = browser.page_source

        # Parse the HTML content
        soup = BeautifulSoup(page_source, 'html.parser')

        data = {}

        # Locate the main content area
        main = soup.select_one('#body-wrapper')
        data['source_name'] = 'dubizzle'
        data['source'] = url
        data['Listing Type'] = listing_type
        if main:
            # Extract span elements under the main section

            overview = main.find('div', {'aria-label': 'Overview'})
            price = overview.find('span', {'aria-label': 'Price'})
            creation_date = overview.find('span', {'aria-label': 'Creation date'})
            data['price'] = price.get_text(strip=True)
            data['creation date'] = creation_date.get_text(strip=True)
            data['extraction date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Find highlighted section
            highlighted = main.find('div', {'class': 'ab0d2d79'})  # Replace with actual class
            if highlighted:
                highlights = highlighted.find_all('span')
                current_label = None

                for child in highlights:
                    text = child.get_text(strip=True)
                    if current_label is None:
                        current_label = text
                    else:
                        data[current_label] = text
                        current_label = None

            # Find details section
            details = soup.find('div', {'aria-label': 'Details'})  # Replace with actual id or class
            if details:
                all_children = details.find_all('span')
                current_label = None

                for child in all_children:
                    text = child.get_text(strip=True)
                    if current_label is None:
                        current_label = text
                    else:
                        data[current_label] = text
                        current_label = None

            feature = main.find('div', {'aria-label': 'Features'})
            if feature:
                features = feature.find_all('span')
                for span in features:
                    text = span.get_text(strip=True)  # Strip leading/trailing whitespace
                    if text:
                        data[text] = 1

            # Find location
            location = soup.find('span', {'aria-label': 'Location'})
            if location:
                data['location'] = location.get_text(strip=True)

            if feature:
                see_all = feature.find(attrs={'class': 'a1c1940e'})
                if see_all is not None:
                    try:
                        s_features = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Features']")))
                        see_all = s_features.find_element(By.XPATH, ".//span[text()='See All']")
                        browser.execute_script("arguments[0].click();", see_all)

                        dialog = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@aria-label='Dialog']")))
                        all_children = dialog.find_elements(By.CLASS_NAME, '_7fb174a1')
                        for cc in all_children:
                            data[cc.text] = '1'
                    except NoSuchElementException:
                        print("Element 'See All' not found, skipping this part.")

            location = browser.find_element(By.XPATH, "//div[@aria-label='Location']")
            button = location.find_element(By.XPATH, ".//div//div//div//div//button")
            browser.execute_script("arguments[0].click();", button)
            # button.click()

            dialog = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, '_721fe53c')))
            image_src = dialog.get_attribute('src')
            img_src_arr = image_src.rsplit('/')
            coord = img_src_arr[8].rsplit(',')
            data['X'] = coord[0]
            data['Y'] = coord[1]
            data['img_src'] = image_src

            # Find description
            description = main.find('div', {'aria-label': 'Description'})
            span = description.find('span')
            data['description'] = span.text.strip()
        print(data)
    # except ConnectionError as e:
    #     print("Error in url ", url)
    #     print(f"Connection error occurred: {e}")
    # except Timeout as e:
    #     print("Error in url ", url)
    #     print(f"Timeout error occurred: {e}")
    # except TooManyRedirects as e:
    #     print("Error in url ", url)
    #     print(f"Too many redirects: {e}")
    # except RequestException as e:
    #     print("Error in url ", url)
    #     print(f"An error occurred: {e}")
    # except Exception as e:
    #     print("Error in url ", url)
    #     print(f"An error occurred: {e}")
    # except TimeoutException as e:
    #     print("Error in url ", url)
    #     print("An error occurred: Timeout while loading the page or finding elements.")
    finally:
        browser.close()
        browser.quit()
    return data

In [ ]:
# def scraping_data(path, file_name):

#     # Load the CSV file into a DataFrame
#     urls = pd.read_csv(path + file_name)
#     print(len(urls))

#     all_data = []

#     for index, url in urls.iterrows():
#         print(f"{index} - {url['URL']}")
#         page_data = scrape_page(url['URL'])
#         all_data.append(page_data)

#     # Create a DataFrame
#     df = pd.DataFrame(all_data)

#     # Filter out None values
#     filtered_data = [item for item in all_data if item is not None]

#     # Create DataFrame
#     df = pd.DataFrame(filtered_data)

#     csv_file_path = f'{path}data_{file_name}'
#     df.to_csv(csv_file_path, index=False)

In [ ]:
import pandas as pd
import concurrent.futures


def scraping_data(path, file_name, type):
    # Load the CSV file into a DataFrame
    urls = pd.read_csv(path + file_name)
    print(len(urls))

    all_data = []

    def process_url(url, type):
        print(f"Processing {url}")
        return scrape_page(url, type)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the process_url function to the list of URLs
        future_to_url = {executor.submit(process_url, url['URL'], type): url for index, url in urls.iterrows()}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                page_data = future.result()
                all_data.append(page_data)
            except Exception as exc:
                print(f"{url['URL']} generated an exception: {exc}")

    # Filter out None values
    filtered_data = [item for item in all_data if item is not None]

    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    csv_file_path = f'{path}data_{file_name}'
    df.to_csv(csv_file_path, index=False)

In [ ]:
# Replace the path with your file's path
file_path_rent = '/content/drive/MyDrive/university M2/M2 final project/dubizzle real estate data/'
file_path_sale = '/content/drive/MyDrive/university M2/M2 final project/dubizzle real estate data/'
scraping_data(file_path_rent, 'urls_to_be_scraped_rent_3.csv', 'Rent')
# scraping_data(file_path_sale, 'urls_to_be_scraped_sale_3.csv', 'Sell')

7335
Processing /ad/mansourieh-furnished-40m2-60m2-terrace-247-electricity-new-ID115790406.html
Processing /ad/modern-apartment-calm-area-247-1-pkg-ID115979910.html
Processing /ad/fully-furnished-in-rawche-prime-150sq-3-bedrooms-jnr-276-ID115898343.htmlProcessing /ad/ra22-1098-apartment-for-rent-in-beirut-verdun-350m-2000-cash-ID114525731.html
Processing /ad/apartment-for-rent-in-verdun%D8%B4%D9%82%D8%A9-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1-%D8%A8%D9%81%D8%B1%D8%AF%D8%A7%D9%86-cpbtf02-ID116019915.html
Processing /ad/apartment-for-rent-in-bauchrieh-%D8%B4%D9%82%D8%A9-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1-%D9%81%D9%8A-%D8%A7%D9%84%D8%A8%D9%88%D8%B4%D8%B1%D9%8A%D8%A9-db-ID115998589.html

{'source_name': 'dubizzle', 'source': 'https://www.dubizzle.com.lb//ad/furnished-apartment-in-ajaltoun-%D8%B4%D9%82%D8%A9-%D9%85%D9%81%D8%B1%D9%88%D8%B4%D8%A9-%D9%81%D9%8A-%D8%B9%D8%AC%D9%84%D8%AA%D9%88%D9%86-ID116009462.html', 'Listing Type': 'Rent', 'price': 'USD 350', 'creation date': '3 wee